## Preprocessing

In [62]:
# Third attempt at optimization - trying to improve nn model performance

# gonna try another path here - Going to abandon what I did with classification and income_amt.  Just going to dummy those fields
# Also, switching activation function to leaky relu.

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import h5py
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [63]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [65]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()

print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [66]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

display(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [67]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# changing threshold back to 200

# set threshold at 200, then flag any value under that amount
threshold = 200
application_types_to_replace = application_type_counts[application_type_counts < threshold].index

# replace less frequent types with "Other"
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, 'Other', inplace=True)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [68]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

display(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [69]:
# You may find it helpful to look at CLASSIFICATION value counts >1
display(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [70]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# changing threshold back to 1000

# set threshold at 1000
threshold = 1000
classifications_to_replace = classification_counts[classification_counts < threshold].index

# replace less frequent types with Other
application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other', inplace=True)

# replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [71]:
# Convert categorical data to numeric with `pd.get_dummies`

# Going to try dummying more fields this time - as opposed to using their numerical values for some features.

application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [72]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [53]:
# added cell - output csvs to troubleshoot

# skipping this cell for optimization tries - don't need it anymore

# Convert X_train and y_train to DataFrames
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)

# Save X_train and y_train to CSV files
X_train_df.to_csv('X_train.csv', index=False)
y_train_df.to_csv('y_train.csv', index=False)

## Compile, Train and Evaluate the Model

In [78]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# adding a layer, as well as increasing number of nodes in existing layers

hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 24
hidden_nodes_layer4 = 18

nn = tf.keras.models.Sequential()

# first layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=43, activation=LeakyReLU(alpha=0.1)))

# second layer
nn.add(Dense(units=hidden_nodes_layer2, input_dim=43, activation=LeakyReLU(alpha=0.1)))

# third layer
nn.add(Dense(units=hidden_nodes_layer3, input_dim=43, activation=LeakyReLU(alpha=0.1)))

# fourth layer
nn.add(Dense(units=hidden_nodes_layer3, input_dim=43, activation=LeakyReLU(alpha=0.1)))

# output layer
nn.add(Dense(units=1, activation='sigmoid'))

In [79]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [80]:
# convert data types to float32
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

# convert X_test and y_test data types to float32
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [81]:
# Train the model
history = nn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
858/858 [==============================] - 4s 3ms/step - loss: 17617.5332 - accuracy: 0.4911 - val_loss: 3065.2324 - val_accuracy: 0.5341
Epoch 2/10
858/858 [==============================] - 3s 4ms/step - loss: 4337.0396 - accuracy: 0.4962 - val_loss: 23831.1426 - val_accuracy: 0.4659
Epoch 3/10
858/858 [==============================] - 2s 3ms/step - loss: 12055.6191 - accuracy: 0.4873 - val_loss: 410.7556 - val_accuracy: 0.5016
Epoch 4/10
858/858 [==============================] - 2s 3ms/step - loss: 1398.4862 - accuracy: 0.4948 - val_loss: 2205.8943 - val_accuracy: 0.4659
Epoch 5/10
858/858 [==============================] - 2s 3ms/step - loss: 1309.8550 - accuracy: 0.4939 - val_loss: 105.5645 - val_accuracy: 0.5341
Epoch 6/10
858/858 [==============================] - 2s 3ms/step - loss: 6913.1279 - accuracy: 0.5019 - val_loss: 9621.5117 - val_accuracy: 0.5341
Epoch 7/10
858/858 [==============================] - 3s 4ms/step - loss: 2860.4688 - accuracy: 0.5020 - val_lo

In [82]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.7592 - accuracy: 0.5331 - 280ms/epoch - 1ms/step
Loss: 0.759230375289917, Accuracy: 0.533090353012085


In [83]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [84]:
# added cell - in-depth model stats and confusion matrix

# load model from HDF5 file
model_path = "AlphabetSoupCharity_Optimization3.h5"
model = load_model(model_path)

# make predictions on test data
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# assuming y_test and y_pred are one-dimensional arrays
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
target_names = ["Class 0", "Class 1"]  # Update class names as needed
class_report = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:")
print(class_report)

215/215 [==============================] - 1s 2ms/step
Confusion Matrix:
[[3196    0]
 [3664    0]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.47      1.00      0.64      3196
     Class 1       0.00      0.00      0.00      3664

    accuracy                           0.47      6860
   macro avg       0.23      0.50      0.32      6860
weighted avg       0.22      0.47      0.30      6860



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
